# Extended Spotify Analysis 
**Data Mining Course**

This project analyzes personal Spotify streaming history using various data mining techniques:
- K-Means Clustering for activity patterns
- Apriori Algorithm for artist association rules
- Random Forest Regression for future prediction
- Statistical analysis and anomaly detection

## 1. Import Libraries
Import all necessary libraries for data processing, analysis, and visualization

In [ ]:
# Data manipulation
import json
import numpy as np
import pandas as pd
from datetime import datetime
from pathlib import Path

# Visualization
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

# Machine Learning
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, silhouette_score, davies_bouldin_score

# Association Rules
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

# Statistics
from scipy import stats as scipy_stats

# Settings
import warnings
warnings.filterwarnings('ignore')
pio.templates.default = "plotly_dark"

## 2. Load Data from JSON Files
Load and combine all Spotify streaming history JSON files

In [ ]:
# List of all JSON files containing streaming history
json_files = [
    'Streaming_History_Audio_2023_0.json',
    'Streaming_History_Audio_2023_1.json',
    'Streaming_History_Audio_2023_2.json',
    'Streaming_History_Audio_2023-2024_3.json',
    'Streaming_History_Audio_2024_4.json',
    'Streaming_History_Audio_2024_5.json',
    'Streaming_History_Audio_2024_6.json',
    'Streaming_History_Audio_2024_7.json',
    'Streaming_History_Audio_2024_8.json',
    'Streaming_History_Audio_2024_9.json',
    'Streaming_History_Audio_2024-2025_10.json',
    'Streaming_History_Audio_2025_11.json',
    'Streaming_History_Audio_2025_12.json',
    'Streaming_History_Audio_2025_13.json',
    'Streaming_History_Audio_2025_14.json'
]

# Initialize empty list to store all records
all_records = []

# Read each JSON file and extend the records list
for file_name in json_files:
    with open(file_name, encoding='utf-8') as file:
        data = json.load(file)
        all_records.extend(data)

# Create DataFrame from all records
df = pd.DataFrame(all_records)

print(f"Total raw records loaded: {len(df):,}")
print(f"Columns in dataset: {list(df.columns)}")

## 3. Data Preprocessing
Clean and prepare the data for analysis

In [ ]:
# Convert timestamp to datetime format. 2 different column names in different files
if 'ts' in df.columns:
    df['ts'] = pd.to_datetime(df['ts'])
else:
    df['ts'] = pd.to_datetime(df['endTime'])

# Standardize column names
if 'master_metadata_album_artist_name' not in df.columns:
    df['master_metadata_album_artist_name'] = df['artistName']
if 'master_metadata_track_name' not in df.columns:
    df['master_metadata_track_name'] = df['trackName']
if 'ms_played' not in df.columns:
    df['ms_played'] = df['msPlayed']

# Filter: Keep only tracks played for at least 30 seconds, to skip accidental plays
df = df[df['ms_played'] >= 30000].copy()

# Create new features for analysis
df['date'] = df['ts'].dt.date                   
df['hour'] = df['ts'].dt.hour                  
df['month'] = df['ts'].dt.to_period('M')        
df['day_name'] = df['ts'].dt.day_name()         
df['weekday'] = df['ts'].dt.dayofweek           
df['is_weekend'] = df['weekday'] >= 5            
df['hours_played'] = df['ms_played'] / 3600000   

print(f"\nRecords after filtering (>30s plays): {len(df):,}")
print(f"Date range: {df['ts'].min().date()} to {df['ts'].max().date()}")
print(f"\nDataset shape: {df.shape}")
print("\nFirst few rows:")
df.head()

## 4. Exploratory Data Analysis (EDA)
Basic statistics and insights about listening habits

In [ ]:
# Calculate basic statistics
total_listening_hours = df['hours_played'].sum()
unique_artists_count = df['master_metadata_album_artist_name'].nunique()
unique_tracks_count = df['master_metadata_track_name'].nunique()
most_played_artist = df['master_metadata_album_artist_name'].mode()[0]
most_played_track = df['master_metadata_track_name'].mode()[0]
most_played_track_count = df['master_metadata_track_name'].value_counts().iloc[0] # value counts: how many times repeated results as a list. 
# different from mode which gives only the most frequent value.

# Display summary statistics
print("LISTENING STATISTICS SUMMARY")
print("-" * 50)
print(f"Total Listening Time: {total_listening_hours:.2f} hours") #.2f only last two difits 
print(f"Unique Artists: {unique_artists_count}")
print(f"Unique Tracks: {unique_tracks_count}")
print(f"Most Played Artist: {most_played_artist}")
print(f"Most Played Track: {most_played_track} ({most_played_track_count} plays)")

### 4.1 Weekly Activity Heatmap
Visualize listening patterns by day of week and hour

In [ ]:
# Define day order for proper sorting
days_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

# Group by day and hour to count plays
heatmap_data = df.groupby(['day_name', 'hour']).size().reset_index(name='play_count') #resets index after groupby, and makes it 'play_count'
# output will be a dataframe with three columns: day_name, hour, play_count indexed by play_count

# Sort days properly
heatmap_data['day_name'] = pd.Categorical(heatmap_data['day_name'], categories=days_order, ordered=True)

# Pivot to create matrix for heatmap
heatmap_matrix = heatmap_data.pivot(index='day_name', columns='hour', values='play_count').fillna(0)

# Create heatmap visualization
fig_heatmap = px.imshow(heatmap_matrix,
                        labels=dict(x="Hour of Day", y="Day of Week", color="Plays"),
                        title='Weekly Listening Pattern Heatmap',
                        color_continuous_scale='Greens')
display(fig_heatmap)

print("Heatmap shows when you listen to music the most during the week.")

### 4.2 Top 10 Most Played Tracks

In [ ]:
# Get top 10 tracks
top_10_tracks = df['master_metadata_track_name'].value_counts().head(10).reset_index()
top_10_tracks.columns = ['Track Name', 'Play Count']

# Create horizontal bar chart
fig_top10 = px.bar(top_10_tracks, 
                   x='Play Count', 
                   y='Track Name', 
                   orientation='h',
                   title='Top 10 Most Played Tracks of All Time',
                   color='Play Count', 
                   color_continuous_scale='Greens')

# Reverse y-axis to show highest at top
fig_top10.update_layout(yaxis=dict(autorange="reversed"))
fig_top10.show()

### 4.3 Trend Analysis: Forgotten vs Rising Artists
Identify artists you used to listen to but stopped, and new favorites

In [ ]:
# Define time periods: in my case it is last 90 days vs before that
last_date = df['ts'].max()
cutoff_date = last_date - pd.Timedelta(days=90) #TimeDelta is used to subtract days from a date.

# Split data into past and recent
past_data = df[df['ts'] < cutoff_date]
recent_data = df[df['ts'] >= cutoff_date]

# Calculate average plays per month for each artist in both periods
past_months = len(past_data['month'].unique()) or 1 # or 1 is used to avoid division by zero errors.
recent_months = len(recent_data['month'].unique()) or 1

past_artist_stats = past_data.groupby('master_metadata_album_artist_name').size() / past_months
recent_artist_stats = recent_data.groupby('master_metadata_album_artist_name').size() / recent_months

# Forgotten Artists: played a lot before (>5/month) but rarely now (<1/month)
forgotten_artists = past_artist_stats[
    (past_artist_stats > 5) & 
    (recent_artist_stats.reindex(past_artist_stats.index).fillna(0) < 1)
].sort_values(ascending=False).head(5)

# Rising Artists: playing a lot now (>10/month) and at least 2x more than before
rising_artists = recent_artist_stats[
    (recent_artist_stats > 10) & 
    (recent_artist_stats > past_artist_stats.reindex(recent_artist_stats.index).fillna(0) * 2)
].sort_values(ascending=False).head(5)

print("\n📉 FORGOTTEN ARTISTS (used to listen, now don't):")
for artist, plays_per_month in forgotten_artists.items():
    print(f"  • {artist}: {plays_per_month:.1f} plays/month in the past")

print("\n📈 RISING ARTISTS (new favorites):")
for artist, plays_per_month in rising_artists.items():
    print(f"  • {artist}: {plays_per_month:.1f} plays/month recently")

## 5. K-Means Clustering Analysis
Group hours of the day by listening activity patterns

### 5.1 Prepare Data for Clustering

In [ ]:
# Aggregate data by hour of day
hourly_stats = df.groupby('hour').agg({
    'ms_played': 'sum',                        # Total time played
    'master_metadata_track_name': 'count'      # Number of plays
}).rename(columns={'master_metadata_track_name': 'play_count'})

# Convert to hours for better interpretation
hourly_stats['hours_played'] = hourly_stats['ms_played'] / 3600000

# Prepare features for clustering: [hours_played, play_count]
X_clustering = hourly_stats[['hours_played', 'play_count']].values

# Standardize features (important for K-Means)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_clustering)

print("Data prepared for clustering:")
print(hourly_stats.head())

### 5.2 Elbow Method to Find Optimal Number of Clusters

In [ ]:
# Calculate WCSS (Within-Cluster Sum of Squares) for k=1 to k=10
wcss_values = []
k_range = range(1, 11)

for k in k_range:
    kmeans_temp = KMeans(n_clusters=k, init='k-means++', random_state=42, n_init=10)
    kmeans_temp.fit(X_scaled)
    wcss_values.append(kmeans_temp.inertia_)

# Plot elbow curve
fig_elbow = px.line(x=list(k_range), y=wcss_values, markers=True,
                    title='Elbow Method for Optimal K',
                    labels={'x': 'Number of Clusters (k)', 'y': 'WCSS (Inertia)'})
fig_elbow.add_annotation(x=3, y=wcss_values[2],
                         text="Elbow Point (k=3)",
                         showarrow=True, arrowhead=2)
fig_elbow.show()

print("The 'elbow' in the graph suggests the optimal number of clusters.")
print("Based on the curve, k=3 appears to be a good choice.")

### 5.3 Apply K-Means with k=3

In [ ]:
# Apply K-Means clustering with k=3
optimal_k = 3
kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
hourly_stats['cluster'] = kmeans.fit_predict(X_scaled)

# Assign meaningful names to clusters based on play_count
cluster_means = hourly_stats.groupby('cluster')['play_count'].mean().sort_values()
cluster_labels = {
    cluster_means.index[0]: 'Low Activity',
    cluster_means.index[1]: 'Medium Activity',
    cluster_means.index[2]: 'High Activity'
}
hourly_stats['cluster_label'] = hourly_stats['cluster'].map(cluster_labels)

# Calculate clustering quality metrics
silhouette = silhouette_score(X_scaled, hourly_stats['cluster'])
davies_bouldin = davies_bouldin_score(X_scaled, hourly_stats['cluster'])

print(f"\nClustering Quality Metrics:")
print(f"Silhouette Score: {silhouette:.3f} (higher is better, range: -1 to 1)")
print(f"Davies-Bouldin Index: {davies_bouldin:.3f} (lower is better)")

# Show which hours belong to which cluster
print("\nHours grouped by activity level:")
for label in ['Low Activity', 'Medium Activity', 'High Activity']:
    hours = hourly_stats[hourly_stats['cluster_label'] == label].index.tolist()
    print(f"{label}: Hours {hours}")

### 5.4 Visualize Clusters

In [ ]:
# Create scatter plot with clusters
fig_kmeans = px.scatter(hourly_stats.reset_index(),
                        x='hour',
                        y='play_count',
                        color='cluster_label',
                        size='hours_played',
                        title='Hourly Activity Clusters (K-Means)',
                        labels={'hour': 'Hour of Day', 'play_count': 'Number of Plays'},
                        hover_data=['hours_played'])
fig_kmeans.show()

print("Each point represents an hour. Colors show activity clusters.")

## 6. Association Rules Mining (Apriori Algorithm)
Discover which artists are frequently listened to together

In [ ]:
# Data preparation
# Remove rows with missing artist names
df_clean = df.dropna(subset=['master_metadata_album_artist_name'])

# Filter: Keep only artists listened to on at least 10 different days
# This improves performance and focuses on regular artists
artist_frequency = df_clean.groupby('master_metadata_album_artist_name')['date'].nunique()
frequent_artists = artist_frequency[artist_frequency >= 10].index
df_filtered = df_clean[df_clean['master_metadata_album_artist_name'].isin(frequent_artists)]

# Create transactions: group artists by date
# Each date is a transaction containing all artists listened that day
transactions = df_filtered.groupby('date')['master_metadata_album_artist_name'].apply(
    lambda x: list(set([str(artist) for artist in x if str(artist) != 'nan']))
).tolist()

print(f"Total transactions (days): {len(transactions)}")
print(f"Artists considered: {len(frequent_artists)}")

# Initialize variables for results
association_rules_df = pd.DataFrame()
fig_apriori = None
top_artist_pairs = pd.DataFrame()

if len(transactions) > 0:
    # Transform transactions into binary matrix format
    te = TransactionEncoder()
    te_array = te.fit_transform(transactions)
    df_encoded = pd.DataFrame(te_array, columns=te.columns_)
    
    # Apply Apriori algorithm
    # min_support=0.02 means artist pair must appear in at least 2% of days
    frequent_itemsets = apriori(df_encoded, min_support=0.02, use_colnames=True)
    
    if not frequent_itemsets.empty:
        print(f"Frequent itemsets found: {len(frequent_itemsets)}")
        
        # Generate association rules
        rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1.0)
        association_rules_df = rules.sort_values('lift', ascending=False)
        
        if not association_rules_df.empty:
            # Convert frozenset to string for display
            association_rules_df['antecedents_str'] = association_rules_df['antecedents'].apply(
                lambda x: ', '.join(list(x)))
            association_rules_df['consequents_str'] = association_rules_df['consequents'].apply(
                lambda x: ', '.join(list(x)))
            
            print(f"Association rules generated: {len(association_rules_df)}\n")
            
            # Visualization: Support vs Confidence, sized by Lift
            fig_apriori = px.scatter(association_rules_df,
                                     x='support',
                                     y='confidence',
                                     size='lift',
                                     color='lift',
                                     hover_data=['antecedents_str', 'consequents_str'],
                                     title='Artist Association Rules (Apriori)',
                                     labels={'support': 'Support', 
                                            'confidence': 'Confidence', 
                                            'lift': 'Lift'},
                                     color_continuous_scale='Viridis')
            fig_apriori.show()
            
            # Create top pairs table
            total_days = len(transactions)
            association_rules_df['co_listen_count'] = (association_rules_df['support'] * total_days).astype(int)
            
            top_artist_pairs = association_rules_df[
                ['antecedents_str', 'consequents_str', 'co_listen_count']
            ].head(10)
            top_artist_pairs.columns = ['Artist 1', 'Artist 2', 'Days Listened Together']
            
            print("\nTop 10 Artist Pairs Listened Together:")
            print(top_artist_pairs.to_string(index=False))
            
            # Explain metrics
            print("\n📊 Metrics Explanation:")
            print("  • Support: How often the artist pair appears together")
            print("  • Confidence: Probability of listening to Artist 2 when listening to Artist 1")
            print("  • Lift: How much more likely to listen to both together vs separately")
            print("    (Lift > 1 means positive association)")
        else:
            print("No association rules found.")
    else:
        print("No frequent itemsets found. Try lowering min_support.")
else:
    print("No transaction data available.")

## 7. Statistical Tests and Anomaly Detection

### 7.1 T-Test: Weekday vs Weekend Listening

In [ ]:
# Separate weekday and weekend listening hours
weekday_listening = df[~df['is_weekend']]['hours_played']
weekend_listening = df[df['is_weekend']]['hours_played']

# Perform independent t-test
t_statistic, p_value = scipy_stats.ttest_ind(weekday_listening, weekend_listening, nan_policy='omit')

# Display results
print("\n" + "="*50)
print("T-TEST: Weekday vs Weekend Listening Habits")
print("="*50)
print(f"Weekday Average: {weekday_listening.mean():.4f} hours per play")
print(f"Weekend Average: {weekend_listening.mean():.4f} hours per play")
print(f"\nT-statistic: {t_statistic:.4f}")
print(f"P-value: {p_value:.4f}")
print(f"\nConclusion: ", end="")
if p_value < 0.05:
    print("There IS a statistically significant difference (p < 0.05)")
else:
    print("There is NO statistically significant difference (p >= 0.05)")
print("="*50)

### 7.2 Anomaly Detection Using Z-Score

In [ ]:
# Count plays per day
daily_play_counts = df.groupby('date').size()

# Calculate Z-scores (how many standard deviations from mean)
mean_plays = daily_play_counts.mean()
std_plays = daily_play_counts.std()
z_scores = (daily_play_counts - mean_plays) / std_plays

# Identify anomalies: Z-score > 3 or < -3 (outliers)
anomaly_threshold = 3
anomaly_days = daily_play_counts[z_scores.abs() > anomaly_threshold]

print(f"\nAnomaly Detection Results:")
print(f"Total days analyzed: {len(daily_play_counts)}")
print(f"Anomalous days detected: {len(anomaly_days)}")
print(f"Average daily plays: {mean_plays:.1f}")

if not anomaly_days.empty:
    print(f"\nMost extreme anomaly:")
    max_anomaly_date = anomaly_days.idxmax()
    print(f"  Date: {max_anomaly_date}")
    print(f"  Plays: {anomaly_days.max()} (Z-score: {z_scores[max_anomaly_date]:.2f})")

# Visualize daily plays with anomalies highlighted
fig_anomaly = go.Figure()

# Normal days
fig_anomaly.add_trace(go.Scatter(
    x=daily_play_counts.index,
    y=daily_play_counts.values,
    mode='lines',
    name='Daily Plays',
    line=dict(color='lightblue')
))

# Anomaly days
if not anomaly_days.empty:
    fig_anomaly.add_trace(go.Scatter(
        x=anomaly_days.index,
        y=anomaly_days.values,
        mode='markers',
        name='Anomalies',
        marker=dict(color='red', size=10, symbol='x')
    ))

fig_anomaly.update_layout(
    title='Daily Listening with Anomaly Detection',
    xaxis_title='Date',
    yaxis_title='Number of Plays'
)
fig_anomaly.show()

## 8. Time Series Forecasting with Random Forest
Predict future monthly listening trends

### 8.1 Prepare Time Series Data

In [ ]:
# Aggregate plays by month
monthly_play_counts = df.groupby('month').size()

# Create time index (0, 1, 2, 3, ... for each month)
X_time = np.arange(len(monthly_play_counts)).reshape(-1, 1)
y_plays = monthly_play_counts.values

# Add seasonality features using sine and cosine
# This helps model capture monthly patterns (e.g., summer vs winter)
month_numbers = np.array([period.month for period in monthly_play_counts.index]).reshape(-1, 1)

# Combine features: [time_index, sin(month), cos(month)]
X_features = np.hstack([
    X_time,
    np.sin(2 * np.pi * month_numbers / 12),  # Seasonal sine
    np.cos(2 * np.pi * month_numbers / 12)   # Seasonal cosine
])

print(f"Training data: {len(monthly_play_counts)} months")
print(f"Features per month: {X_features.shape[1]} (time + seasonality)")

### 8.2 Train Random Forest Model

In [ ]:
# Initialize and train Random Forest Regressor
rf_model = RandomForestRegressor(
    n_estimators=100,      # Number of trees
    random_state=42,       # For reproducibility
    n_jobs=-1              # Use all CPU cores
)

print("Training Random Forest model...")
rf_model.fit(X_features, y_plays)

# Evaluate model on training data
y_predicted = rf_model.predict(X_features)
r2 = r2_score(y_plays, y_predicted)
rmse = np.sqrt(mean_squared_error(y_plays, y_predicted))

print(f"\nModel Performance:")
print(f"R² Score: {r2:.3f} (1.0 is perfect fit)")
print(f"RMSE: {rmse:.2f} plays")

# Feature importance
feature_names = ['Time Trend', 'Seasonality (sin)', 'Seasonality (cos)']
importances = rf_model.feature_importances_
print(f"\nFeature Importance:")
for name, importance in zip(feature_names, importances):
    print(f"  {name}: {importance:.3f}")

### 8.3 Generate Future Predictions

In [ ]:
# Predict next 6 months
forecast_months = 6
last_time_index = X_time[-1][0]
last_date = monthly_play_counts.index[-1].to_timestamp()

# Create future time indices
future_time_indices = np.array([[last_time_index + i] for i in range(1, forecast_months + 1)])

# Calculate future month numbers (wraps around: 12 -> 1)
last_month_num = month_numbers[-1][0]
future_month_numbers = np.array([[(last_month_num + i - 1) % 12 + 1] 
                                 for i in range(1, forecast_months + 1)])

# Create future feature matrix with seasonality
X_future = np.hstack([
    future_time_indices,
    np.sin(2 * np.pi * future_month_numbers / 12),
    np.cos(2 * np.pi * future_month_numbers / 12)
])

# Make predictions
future_predictions = rf_model.predict(X_future)

# Generate future month labels
future_month_labels = []
for i in range(1, forecast_months + 1):
    future_date = last_date + pd.DateOffset(months=i)
    future_month_labels.append(future_date.strftime('%Y-%m'))

# Display predictions
print("\n" + "="*50)
print("FORECAST: Next 6 Months")
print("="*50)
for month, prediction in zip(future_month_labels, future_predictions):
    print(f"{month}: {int(prediction)} predicted plays")
print("="*50)

### 8.4 Visualize Historical Data and Forecast

In [ ]:
# Prepare data for visualization
historical_months = [str(period) for period in monthly_play_counts.index]

# Create figure
fig_forecast = go.Figure()

# 1. Actual historical data (bars)
fig_forecast.add_trace(go.Bar(
    x=historical_months,
    y=y_plays,
    name='Actual Plays',
    marker_color='#1DB954'  # Spotify green
))

# 2. Model fit on historical data (orange line)
fig_forecast.add_trace(go.Scatter(
    x=historical_months,
    y=y_predicted,
    name='Model Fit',
    line=dict(color='orange', width=2),
    mode='lines'
))

# 3. Future predictions (red dashed line)
# Connect last historical point to forecast for smooth transition
forecast_x = [historical_months[-1]] + future_month_labels
forecast_y = [y_predicted[-1]] + list(future_predictions)

fig_forecast.add_trace(go.Scatter(
    x=forecast_x,
    y=forecast_y,
    name='Forecast',
    line=dict(color='red', width=3, dash='dot'),
    mode='lines+markers'
))

# Layout
fig_forecast.update_layout(
    title='Monthly Listening History and 6-Month Forecast',
    xaxis_title='Month',
    yaxis_title='Number of Plays',
    hovermode='x unified'
)

fig_forecast.show()